## 1) Базовое определение и пример

Замыкание — функция вместе с окружением её свободных переменных. Пример фабрики функций:

In [5]:
def make_multiplier(factor):
    def multiply(x):
        return x * factor
    return multiply

double = make_multiplier(2)
triple = make_multiplier(3)

print(double(10))
print(triple(10))
print('double.__closure__:', double.__closure__)
print('double.__code__.co_freevars:', double.__code__.co_freevars)

20
30
double.__closure__: (<cell at 0x1082c0c70: int object at 0x1043169d8>,)
double.__code__.co_freevars: ('factor',)


## 2) Инспекция замыкания (`__closure__`, `co_freevars`) и получение значений

Используем inspect.getclosurevars и прямой доступ к cell_contents.

In [21]:
import inspect

def outer(x):
    y = x + 1
    def inner(z):
        return y + z
    return inner

f = outer(10)
print('co_freevars:', f.__code__.co_freevars)
print('closure cells:', f.__closure__)
print('closure values:', tuple(c.cell_contents for c in f.__closure__))
print('inspect.getclosurevars:', inspect.getclosurevars(f))

co_freevars: ('y',)
closure cells: (<cell at 0x107be4820: int object at 0x104316af8>,)
closure values: (11,)


AttributeError: 'tuple' object has no attribute 'fset'

## 3) `nonlocal` — изменение внешнего состояния внутри вложенной функции

Пример счётчика, использующего nonlocal.

In [7]:
def counter(start=0):
    n = start
    def inc():
        nonlocal n
        n += 1
        return n
    def get():
        return n
    return inc, get

inc, get = counter(5)
print(inc(), inc(), get())

6 7 7


In [18]:
n = 0
def main():
    def test():
        global n
        n += 1
        print(n)


## 4) Поздняя привязка (late binding) при создании функций в цикле


In [27]:
# Что получится?
funcs = []
for i in range(3):
    funcs.append(lambda: i)
print([f() for f in funcs])
def test(): pass
for i in range(3):
    def test(): print(i)
test()


[2, 2, 2]
2


In [28]:






# Решение 1: привязать через аргумент по умолчанию
funcs = []
for i in range(3):
    funcs.append(lambda i=i: i)
print([f() for f in funcs])

# Решение 2: фабрика
def make(i):
    def inner():
        return i
    return inner

funcs = [make(i) for i in range(3)]
print([f() for f in funcs])

[0, 1, 2]
[0, 1, 2]


## 5) Байткод: `dis` — как Python читает/пишет freevars

Посмотрим инструкции для чтения и записи захваченных переменных.

In [4]:
import dis

def outer_write():
    x = 10
    def inner():
        nonlocal x
        x += 1
        return x
    return inner

def outer_read():
    x = 5
    def inner(y):
        return x + y
    return inner

print('--- READ ---')
dis.dis(outer_read().__code__)
print('--- WRITE ---')
dis.dis(outer_write().__code__)

--- READ ---
  --           COPY_FREE_VARS           1

  13           RESUME                   0

  14           LOAD_DEREF               1 (x)
               LOAD_FAST                0 (y)
               BINARY_OP                0 (+)
               RETURN_VALUE
--- WRITE ---
  --           COPY_FREE_VARS           1

   5           RESUME                   0

   7           LOAD_DEREF               0 (x)
               LOAD_CONST               1 (1)
               BINARY_OP               13 (+=)
               STORE_DEREF              0 (x)

   8           LOAD_DEREF               0 (x)
               RETURN_VALUE


## Упражнения



Реализовать функцию queue(), которая возвращает набор функций (add, pop, show, clear) для управления очередью.
Внутреннее состояние очереди должно храниться в замыкании, без использования глобальных переменных и классов.

In [ ]:
def queue():
    a1 = []
    a2 = []

    def add(x):
        a1.append(x)
        

    def pop():
        if len(a2) > 0:
            return a2.pop()
        else:
            while len(a1) > 0:
                a2.append(a1.pop())
            return a2.pop()

    def clear():
        a1.clear()
        a2.clear()

    def show():
        ans = []
        for i in reversed(a1):
            ans.append(i)
        for i in a2:
            ans.append(i)
        return ans
        

    return add, pop, show, clear

# Использование (пример)

add, pop, show, clear = queue()

add(1)
add(2)
add(3)
pop()
add(4)
print(show())


[4, 3, 2]
